In [13]:
# importing all necessary libraries
import requests
from datetime import datetime
import os


# Extracting Fire Incidents 

In [14]:
# --- Generate the date string ---
today_str = datetime.today().strftime('%Y%m%d')

# --- Define URL components ---
dataset_id = "wr8u-xric"
cache_bust = "1746351254"  # You can randomize this if needed. So far we assume it's static.
url = f"https://data.sfgov.org/api/views/{dataset_id}/rows.csv?fourfour={dataset_id}&cacheBust={cache_bust}&date={today_str}&accessType=DOWNLOAD"

print("Creating variables for URL")
print(f"dataset_id: {dataset_id} \ncache_bust:{cache_bust} \ntoday_str: {today_str}")
print(f"\nURL: {url}")

Creating variables for URL
dataset_id: wr8u-xric 
cache_bust:1746351254 
today_str: 20250510

URL: https://data.sfgov.org/api/views/wr8u-xric/rows.csv?fourfour=wr8u-xric&cacheBust=1746351254&date=20250510&accessType=DOWNLOAD


In [15]:
# --- Define local filename and save path ---
filename = f"Fire_Incidents_{today_str}.csv"
save_path = r"C:\Users\migoc\OneDrive\05 GITHUB\sf-fire\Data"
full_path = os.path.join(save_path, filename)

In [16]:

# --- Download and save the file ---
response = requests.get(url)

if response.status_code == 200:
    with open(full_path, 'wb') as f:
        f.write(response.content)
    print(f"✅ File saved to: {full_path}")
else:
    print(f"❌ Failed to download file. Status code: {response.status_code}")

✅ File saved to: C:\Users\migoc\OneDrive\05 GITHUB\sf-fire\Data\Fire_Incidents_20250510.csv


# Loading to full CSV to Postgres

In [11]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import traceback
import psycopg2

In [ ]:
# --- Database connection settings ---
DB_USER = 'postgres'
DB_PASSWORD = 'password'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'sf_fire_dev'
DB_SCHEMA = 'public'
staging_table_name = 'openSfFireIncident'

In [ ]:
# --- Construct SQLAlchemy connection string ---
connection_string = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

try:
    print(f"🔌 Connecting to database {DB_NAME}")
    engine = create_engine(connection_string)
    connection = engine.connect()
    print("✅ Connected successfully.")
except SQLAlchemyError as e:
    print("❌ Database connection failed.")
    print(traceback.format_exc())
    raise SystemExit()


🔌 Connecting to database...
✅ Connected successfully.


In [17]:
# --- Read and load CSV ---
try:
    print(f"📥 Reading data from: {full_path}")
    df = pd.read_csv(full_path)
    print(f"📊 Loaded DataFrame with {df.shape[0]} rows and {df.shape[1]} columns.")
except Exception as e:
    print("❌ Failed to read CSV file.")
    print(traceback.format_exc())
    raise SystemExit()

📥 Reading data from: C:\Users\migoc\OneDrive\05 GITHUB\sf-fire\Data\Fire_Incidents_20250510.csv


C:\Users\migoc\AppData\Local\Temp\ipykernel_15404\1100132792.py:4: DtypeWarning: Columns (10,12,13,20,29,38,45,46,55,57,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_path)


📊 Loaded DataFrame with 703538 rows and 66 columns.


In [ ]:
# --- Write to PostgreSQL ---
try:
    print(f"🚚 Loading to {DB_SCHEMA}.{staging_table_name}...")
    df.to_sql(
        staging_table_name,
        con=engine,
        schema=DB_SCHEMA,
        if_exists='append',
        index=False
    )
    print(f"✅ Successfully loaded data to table: {staging_table_name}")
except SQLAlchemyError as e:
    print("❌ Failed to load data into PostgreSQL.")
    print(traceback.format_exc())
    raise SystemExit()
finally:
    connection.close()

🚚 Loading to public.openSfFireIncident...


In [ ]:
# # closing the connection 
# connection.close()